# CUDA - GPU - ćwiczenia
**Paweł Kruczkiewicz**


TODO
  - pliki wrzucić tak jak trzeba, żeby się poprawnie kompilowało
  - podzielić logicznie cellki w notebooku
  - dodać odpowiednie kompilacje
  - napisać wnioski

Estimated time < 2h

## Przygotowanie środowiska wykonawczego

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-4ejnqvhv
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-4ejnqvhv
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=26fd59658972fc5e8f4ae1394d4b9c553f03664ad46a75bbd8b3f720d96a7c73
  Stored in directory: /tmp/pip-ephem-wheel-cache-5h_fi45v/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:

!nvidia-smi


Sun Jun 18 18:23:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%cu
#include <iostream>
    int
    main()
{
    std::cout << "Welcome To Informatyka na WIET\n";
    return 0;
}

Welcome To Informatyka na WIET



In [ ]:
%%cu
#include <cstdio>
#include <iostream>

	using namespace std;

__global__ void maxi(int* a, int* b, int n)
{
	int block = 256 * blockIdx.x;
	int max = 0;

	for (int i = block; i < min(256 + block, n); i++) {

		if (max < a[i]) {
			max = a[i];
		}
	}
	b[blockIdx.x] = max;
}

int main()
{

	int n;
	n = 3 >> 2;
	int a[n];

	for (int i = 0; i < n; i++) {
		a[i] = rand() % n;
		cout << a[i] << "\t";
	}

	cudaEvent_t start, end;
	int *ad, *bd;
	int size = n * sizeof(int);
	cudaMalloc(&ad, size);
	cudaMemcpy(ad, a, size, cudaMemcpyHostToDevice);
	int grids = ceil(n * 1.0f / 256.0f);
	cudaMalloc(&bd, grids * sizeof(int));

	dim3 grid(grids, 1);
	dim3 block(1, 1);

	cudaEventCreate(&start);
	cudaEventCreate(&end);
	cudaEventRecord(start);

	while (n > 1) {
		maxi<<<grids, block>>>(ad, bd, n);
		n = ceil(n * 1.0f / 256.0f);
		cudaMemcpy(ad, bd, n * sizeof(int), cudaMemcpyDeviceToDevice);
	}

	cudaEventRecord(end);
	cudaEventSynchronize(end);

	float time = 0;
	cudaEventElapsedTime(&time, start, end);

	int ans[2];
	cudaMemcpy(ans, ad, 4, cudaMemcpyDeviceToHost);

	cout << "The maximum element is : " << ans[0] << endl;

	cout << "The time required : ";
	cout << time << endl;
}


The maximum element is : 0
The time required : 0.00256



## Przygotowanie środeowiska pod laby

In [ ]:
!git clone https://github.com/pkrucz00/parallel-programming.git


Cloning into 'parallel-programming'...
remote: Enumerating objects: 355, done.
remote: Counting objects: 100% (154/154), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 355 (delta 80), reused 116 (delta 75), pack-reused 201
Receiving objects: 100% (355/355), 40.20 MiB | 17.90 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [ ]:
%cd parallel-programming/
!git checkout gpu

/content/parallel-programming
Branch 'gpu' set up to track remote branch 'gpu' from 'origin'.
Switched to a new branch 'gpu'


In [ ]:


!nvidia-smi
!free -g

Sun Jun 18 18:23:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%cd gpu/CUDA-Lab02

/content/parallel-programming/gpu/CUDA-Lab02


In [ ]:
!ls

'1 Reduction'	     '3 Loop Unrolling'     '5 Histogram'   helper_timer.h
'2 Warp Divergence'  '4 Atomic Operations'   exception.h    lab2.ipynb


## Ćwiczenie 1

### Reduction

In [ ]:
%cd "1 Reduction"

/content/parallel-programming/gpu/CUDA-Lab02/1 Reduction


In [ ]:
!ls

analyser.bat	reduction_global.cpp	    reduction_shared.cpp
exception.h	reduction_global_gpu.bat    reduction_shared_kernel.cu
helper_timer.h	reduction_global_kernel.cu
profiler.bat	reduction.h


**Globalna**

In [ ]:
!nvcc -o reduction_global_gpu reduction_global.cpp reduction_global_kernel.cu -arch=sm_50 -allow-unsupported-compiler -I.


In [ ]:
!chmod 777 reduction_global_gpu
!./reduction_global_gpu

Time= 14.344 msec, bandwidth= 4.678633 GB/s
host: 0.996007, device 0.996007


In [ ]:
!nvprof ./reduction_global_gpu

==1311== NVPROF is profiling process 1311, command: ./reduction_global_gpu
Time= 14.762 msec, bandwidth= 4.546142 GB/s
host: 0.996007, device 0.996007
==1311== Profiling application: ./reduction_global_gpu
==1311== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   95.30%  1.41337s      2424  583.07us  452.22us  790.52us  global_reduction_kernel(float*, float*, int, int)
                    3.72%  55.098ms       100  550.98us  545.63us  553.31us  [CUDA memcpy DtoD]
                    0.98%  14.589ms         1  14.589ms  14.589ms  14.589ms  [CUDA memcpy HtoD]
                    0.00%  1.7600us         1  1.7600us  1.7600us  1.7600us  [CUDA memcpy DtoH]
      API calls:   90.06%  1.45614s       101  14.417ms  3.8490us  35.061ms  cudaDeviceSynchronize
                    7.67%  123.98ms         2  61.992ms  172.72us  123.81ms  cudaMalloc
                    1.10%  17.753ms       102  174.05us  15.773us  14.767ms  cudaMe

**Współdzielona**

In [ ]:
!nvcc -o reduction_shared_gpu reduction_shared.cpp reduction_shared_kernel.cu -arch=sm_50 -allow-unsupported-compiler -I.


In [ ]:
!chmod 777 reduction_shared_gpu
!./reduction_shared_gpu

Time= 2.589 msec, bandwidth= 25.918062 GB/s
host: 0.996007, device 0.996007


In [ ]:
!nvprof ./reduction_shared_gpu

==1876== NVPROF is profiling process 1876, command: ./reduction_shared_gpu
Time= 2.972 msec, bandwidth= 22.580601 GB/s
host: 0.996007, device 0.996007
==1876== Profiling application: ./reduction_shared_gpu
==1876== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   77.90%  242.19ms       303  799.32us  4.4160us  2.5105ms  reduction_kernel(float*, float*, unsigned int)
                   17.53%  54.511ms       101  539.71us  538.17us  542.37us  [CUDA memcpy DtoD]
                    4.57%  14.207ms         1  14.207ms  14.207ms  14.207ms  [CUDA memcpy HtoD]
                    0.00%  1.6640us         1  1.6640us  1.6640us  1.6640us  [CUDA memcpy DtoH]
      API calls:   66.47%  295.33ms         1  295.33ms  295.33ms  295.33ms  cudaDeviceSynchronize
                   29.52%  131.18ms         2  65.590ms  163.33us  131.02ms  cudaMalloc
                    3.39%  15.049ms       103  146.11us  4.1480us  14.384ms  cudaMemcp

In [ ]:
%cd ..

/content/parallel-programming/gpu/CUDA-Lab02


**Podsumowanie**

Dzięki programowi `nvprof` można łatwo zauwazyć, że program wykorzystujący globalną pamięć jest zdecydowanie wolniejszy niż jego odpowiednik używający pamięci współdzielonej, ponieważ w pierwszym synchronizacja cudaDeviceSynchronize zajmuje aż ok. 1.5 s, a w drugim raptem 300 ms.

## Ćwiczenie 2


In [ ]:
%cd "2 Warp Divergence"

/content/parallel-programming/gpu/CUDA-Lab02/2 Warp Divergence


In [ ]:
!nvcc -o reduction_interleaving reduction.cpp reduction_kernel_interleaving.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!nvcc -o reduction_sequential reduction.cpp reduction_kernel_sequential.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!chmod 744 reduction_interleaving
!chmod 744 reduction_sequential

!./reduction_interleaving
!./reduction_sequential

Time= 3.024 msec, bandwidth= 22.192085 GB/s
host: 0.996007, device 0.996007
Time= 2.486 msec, bandwidth= 26.990372 GB/s
host: 0.996007, device 0.996007


In [ ]:
%cd ..

/content/parallel-programming/gpu/CUDA-Lab02


**Podsumowanie**

Sekwencyjne wykonywanie ma zdecydowanie lepsze wyniki. Osiąga również lepszy bandwidth.

## Ćwiczenie 3

In [ ]:
%cd "3 Loop Unrolling"

[Errno 2] No such file or directory: '3 Loop Unrolling'
/content/parallel-programming/gpu/CUDA-Lab02/4 Atomic Operations


In [ ]:
!nvcc -o reduction_cg reduction.cpp reduction_cg_kernel.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!nvcc -o reduction_wp reduction.cpp reduction_wp_kernel.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!chmod 744 reduction_cg
!chmod 744 reduction_wp

!./reduction_cg
!./reduction_wp

Time= 0.813 msec, bandwidth= 82.537621 GB/s
host: 0.996007, device 0.996007
Time= 0.809 msec, bandwidth= 82.912888 GB/s
host: 0.996007, device 0.996007


In [ ]:
%cd ..

/content/parallel-programming/gpu/CUDA-Lab02


**Podsumowanie**

Oba programy osiągają porównywanlne wyniki. Oznacza to, że rozwijanie pętli nie zawsze przyśpiesza program.

## Ćwiczenie 4

In [ ]:
%cd "4 Atomic Operations"

/content/parallel-programming/gpu/CUDA-Lab02/4 Atomic Operations


In [ ]:
!nvcc -o reduction reduction.cpp reduction_kernel.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!nvcc -o reduction_blk reduction.cpp reduction_blk_atmc_kernel.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!nvcc -o reduction_wrp reduction.cpp reduction_wrp_atmc_kernel.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!chmod 744 reduction
!chmod 744 reduction_blk
!chmod 744 reduction_wrp

!./reduction
!./reduction_blk
!./reduction_wrp

Time= 35.418 msec, bandwidth= 1.894742 GB/s
host: 0.996007, device 0.995957
Time= 0.807 msec, bandwidth= 83.178024 GB/s
host: 0.996007, device 0.996007
Time= 0.807 msec, bandwidth= 83.177002 GB/s
host: 0.996007, device 0.996008


In [ ]:
%cd ..

/content/parallel-programming/gpu/CUDA-Lab02


**Podsumowanie**

Używanie atomowych operacji daje zdecydowanie lepsze wyniki niż podejście naiwne. Oba sposoby na atomowe operacje są równie dobre.

## Ćwiczenie 5

In [ ]:
%cd "5 Histogram"

[Errno 2] No such file or directory: '5 Histogram'
/content/parallel-programming/gpu/CUDA-Lab02/5 Histogram


In [ ]:
!nvcc -o histo histo.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!nvcc -o reduce reduce.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!chmod 744 histo
!chmod 744 reduce

!./histo
!./reduce

Using device 0:
Tesla T4; global mem: -1344471040B; compute v7.5; clock: 1590000 kHz
Running naive histo
bin 0: count 1
bin 1: count 2
bin 2: count 1
bin 3: count 2
bin 4: count 1
bin 5: count 2
bin 6: count 1
bin 7: count 2
bin 8: count 1
bin 9: count 2
bin 10: count 1
bin 11: count 3
bin 12: count 1
bin 13: count 3
bin 14: count 2
bin 15: count 2
Using device 0:
Tesla T4; global mem: -1344471040B; compute v7.5; clock: 1590000 kHz
Running global reduce
average time elapsed: 0.136013


In [ ]:
!nvcc -o histo2 histo.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!nvcc -o reduce reduce.cu -arch=sm_50 -allow-unsupported-compiler -I.

In [ ]:
!chmod 744 histo2
!chmod 744 reduce

!./histo2
!./reduce

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
bin 126076: count 0
bin 126077: count 0
bin 126078: count 0
bin 126079: count 0
bin 126080: count 0
bin 126081: count 0
bin 126082: count 0
bin 126083: count 0
bin 126084: count 0
bin 126085: count 0
bin 126086: count 0
bin 126087: count 0
bin 126088: count 0
bin 126089: count 0
bin 126090: count 0
bin 126091: count 0
bin 126092: count 0
bin 126093: count 0
bin 126094: count 0
bin 126095: count 0
bin 126096: count 0
bin 126097: count 0
bin 126098: count 0
bin 126099: count 0
bin 126100: count 0
bin 126101: count 0
bin 126102: count 0
bin 126103: count 0
bin 126104: count 0
bin 126105: count 0
bin 126106: count 0
bin 126107: count 0
bin 126108: count 0
bin 126109: count 0
bin 126110: count 0
bin 126111: count 0
bin 126112: count 0
bin 126113: count 0
bin 126114: count 0
bin 126115: count 0
bin 126116: count 0
bin 126117: count 0
bin 126118: count 0
bin 126119: count 0
bin 126120: count 0
bin 126121: count 0
bin 126122: cou

Tworzenie histogramów (przydzielanie do kubełków) dobrze zrównolegla się na kartach graficznych. Zwiększenie liczby kubełków nie wpływa na czas wykonania programu.